In [ ]:
from shapeOptInductor import create_plots, update_plots, rot, gen_mesh9  # noqa: F401
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
thickness = 1e-2

mesh = gen_mesh9(air_gap=1.78e-3, maxh=2e-3)
XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})

Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh, radius=0.02)

In [ ]:
from netgen.geom2d import SplineGeometry


def debug_geometry(
    geo: SplineGeometry, points_names=None, region_names=None, show_regions=True, show_bc=False, offset=1e-3
):
    """Plot the geometry with matplotlib to help debugging"""
    # Plot the points
    points = []
    nb_points = geo.GetNPoints()
    for i in range(nb_points):
        point = geo.GetPoint(i)
        points.append((point[0], point[1]))

    points = np.array(points)

    plt.figure(figsize=(8, 8))
    plt.scatter(points[:, 0], points[:, 1], color="blue")
    if points_names:
        for i in range(len(points_names)):
            plt.text(points[i, 0], points[i, 1], points_names[i], color="blue", fontsize=10)

    # Plot the lines and splines and annotate with region labels
    nb_splines = geo.GetNSplines()
    for i in range(nb_splines):
        spline = geo.GetSpline(i)
        p_start, p_end = spline.StartPoint(), spline.EndPoint()
        plt.plot([p_start[0], p_end[0]], [p_start[1], p_end[1]], color="green")

        # Midpoint for labels
        midpoint = ((p_start[0] + p_end[0]) / 2, (p_start[1] + p_end[1]) / 2)
        if show_regions:
            dx, dy = p_end[0] - p_start[0], p_end[1] - p_start[1]
            normal = np.array([-dy, dx])
            normal_length = np.linalg.norm(normal)
            if normal_length != 0:
                normal = normal / normal_length
            left_domain_position = (midpoint[0] + normal[0] * offset, midpoint[1] + normal[1] * offset)
            right_domain_position = (midpoint[0] - normal[0] * offset, midpoint[1] - normal[1] * offset)
            left_domain_label = spline.leftdom
            right_domain_label = spline.rightdom
            if region_names:
                plt.text(
                    left_domain_position[0],
                    left_domain_position[1],
                    region_names[left_domain_label - 1],
                    color="red",
                    fontsize=10,
                    ha="center",
                )
                plt.text(
                    right_domain_position[0],
                    right_domain_position[1],
                    region_names[right_domain_label - 1],
                    color="red",
                    fontsize=10,
                    ha="center",
                )
            else:
                plt.text(
                    left_domain_position[0],
                    left_domain_position[1],
                    left_domain_label,
                    color="red",
                    fontsize=10,
                    ha="center",
                )
                plt.text(
                    right_domain_position[0],
                    right_domain_position[1],
                    right_domain_label,
                    color="red",
                    fontsize=10,
                    ha="center",
                )
        if show_bc:
            bc_name = geo.GetBCName(spline.bc)
            plt.text(midpoint[0], midpoint[1], bc_name, color="red", fontsize=8, ha="center")

        normal_start = spline.GetNormal(0)
        tangent_start = (-normal_start[1], normal_start[0])
        normal_end = spline.GetNormal(1)
        tangent_end = (-normal_end[1], normal_end[0])
        determinant = tangent_start[0] * tangent_end[1] - tangent_end[0] * tangent_start[1]
        if np.abs(determinant) > 1e-16:
            print("spline3 not yet supported, displaying as a line")

    plt.grid(True)
    plt.axis("equal")
    plt.show()


# help(mesh.ngmesh)
points_names = "p1, p2, p3, p4, p5, p6, p7, p8, p001, p002, p003, p00, p01, p02, p03, p04, p05".split(", ")
region_names = mesh.ngmesh.GetRegionNames(2)
debug_geometry(
    mesh.ngmesh.GetGeometry(), points_names=points_names, region_names=region_names, show_regions=False, show_bc=True
)

In [ ]:
# Frequency
f = 5e4  # Hz
omega = 2 * np.pi * f

# Magnetic
mu0 = 4e-7 * np.pi
mur = 1000
mu_iron = mur * mu0
delta = 0.1
mu_coil = np.exp(-1j * delta) * mu0  #  AC losses in the copper from the imaginary part of the permeability

# Current
nb_turn = 200  # Number of turn in the coil
Is = 1  # Source current intensity
js = nb_turn / 2 * Is / (ngs.Integrate(XiCoil, mesh)) * XiCoil  # Source current density


def magWeakFormComplex(a, a_):
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coil")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")
    lf = a_ * js * ngs.dx("coil")
    return bf, lf


def solveStateComplex(fes):
    a, a_ = fes.TnT()
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f
    K.Assemble()
    F.Assemble()
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    gf.vec.data = Kinv * F.vec
    return gf, Kinv


fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2", complex=True)
a, _ = solveStateComplex(fes)
Draw(ngs.Norm(a), mesh)

In [ ]:
def Inductance(a, mesh):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return 4 * thickness / (2 * Is**2) * ngs.Integrate(rel.real * ngs.Norm(ngs.grad(a)) ** 2, mesh)


def Losses(a, mesh):
    rel = XiCoil / mu_coil
    return 4 * omega / 2 * thickness * ngs.Integrate(rel.imag * ngs.Norm(ngs.grad(a)) ** 2, mesh)


print(f" L = {Inductance(a,mesh) * 1e6 :.2f} µH")
print(f" P_AC = {Losses(a,mesh) :.2f} W")

In [ ]:
L_target = 1e-3


def Constraint(a, mesh):
    return Inductance(a, mesh) - L_target


def CostFunction(a, l, b, constraint, mesh):
    return (Losses(a, mesh) + l * constraint + 0.5 * b * constraint**2).real

In [ ]:
def solveAdjointLosses(a0, Kinv):
    """Solves the adjoint equation for the losses inside the coil"""
    fes = a0.space
    v = fes.TestFunction()
    F = ngs.LinearForm(fes)
    coeff_losses = 4 * omega / 2 * thickness * (XiCoil / mu_coil).imag
    F += -2 * coeff_losses * ngs.InnerProduct(ngs.grad(a0), ngs.grad(v)) * ngs.dx("coil")
    F.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data = Kinv * F.vec
    return gf


def solveAdjointInductance(a0, Kinv):
    """Solves the adjoint equation for the inductance"""
    fes = a0.space
    v = fes.TestFunction()
    F = ngs.LinearForm(fes)
    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    coeff_induc = 4 * thickness / (2 * Is**2) * rel
    F += -2 * coeff_induc * ngs.InnerProduct(ngs.grad(a0), ngs.grad(v)) * ngs.dx
    F.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data = Kinv * F.vec
    return gf


def computeLossesShapeDerivative(VEC_complex, VEC_real, a0, p0):
    """Shape derivative for the energy-based losses inside the coil"""
    X = VEC_complex.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix
    coeff_losses = 4 * omega / 2 * thickness * (1 / mu_coil).imag
    dA = ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans

    dLOmega = ngs.LinearForm(VEC_complex)
    dLOmega += coeff_losses * ngs.InnerProduct(dA * ngs.grad(a0), ngs.grad(a0)) * ngs.dx("coil")
    dLOmega += -js * XiCoil * ngs.div(X) * p0 * ngs.dx
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx

    dLOmega.Assemble()
    dJ = ngs.GridFunction(VEC_real)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def computeInductanceShapeDerivative(VEC_complex, VEC_real, a0, p0):
    """Shape derivative for the inductance"""
    X = VEC_complex.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix
    coeff_induc = 4 * thickness / (2 * Is**2) * rel
    dA = ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans

    dLOmega = ngs.LinearForm(VEC_complex)
    dLOmega += coeff_induc * ngs.InnerProduct(dA * ngs.grad(a0), ngs.grad(a0)) * ngs.dx
    dLOmega += -js * ngs.div(X) * p0 * ngs.dx
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx
    dLOmega.Assemble()

    dJ = ngs.GridFunction(VEC_real)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def computeLossesShapeDerivativeAuto(VEC_complex, VEC_real, a0, p0):  # The Taylor test fail with this one
    """Shape derivative for the energy-based losses inside the coil with DiffShape"""
    bf, lf = magWeakFormComplex(a0, p0)
    coeff_losses = 4 * omega / 2 * thickness * (1 / mu_coil).imag
    f_a0 = coeff_losses * ngs.InnerProduct(ngs.grad(a0), ngs.grad(a0)) * ngs.dx("coil")
    Lagrangian = f_a0 + bf - lf

    # Computation of the shape derivative
    dLOmega = ngs.LinearForm(VEC_complex)
    dLOmega += Lagrangian.DiffShape(VEC_complex.TestFunction())

    ## ... and assembly
    dLOmega.Assemble()
    dJ = ngs.GridFunction(VEC_real)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def computeInductanceShapeDerivativeAuto(VEC_complex, VEC_real, a0, p0):
    """Shape derivative for the inductance with DiffShape"""
    bf, lf = magWeakFormComplex(a0, p0)
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    coeff_induc = 4 * thickness / (2 * Is**2) * rel
    f_a0 = coeff_induc * ngs.InnerProduct(ngs.grad(a0), ngs.grad(a0)) * ngs.dx
    Lagrangian = f_a0 + bf - lf

    # Computation of the shape derivative
    dLOmega = ngs.LinearForm(VEC_complex)
    dLOmega += Lagrangian.DiffShape(VEC_complex.TestFunction())

    ## ... and assembly
    dLOmega.Assemble()
    dJ = ngs.GridFunction(VEC_real)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, l, b):
    """Shape derivative for the cost function"""
    dJOmegaInductance = computeInductanceShapeDerivative(VEC_complex, VEC_real, state, adjoint_inductance)
    constraint = Constraint(state, mesh)
    dJOmega = computeLossesShapeDerivative(VEC_complex, VEC_real, state, adjoint_losses)
    dJOmega.vec.FV().NumPy()[:] += l * dJOmegaInductance.vec.FV().NumPy()[:]
    dJOmega.vec.FV().NumPy()[:] += b * constraint * dJOmegaInductance.vec.FV().NumPy()[:]
    return dJOmega

In [ ]:
mesh = gen_mesh9(air_gap=4e-3, maxh=2e-3)
# mesh = ngs.Mesh("4_optimized_mesh.vol") # Decomment to get optimized shape

fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2", complex=True)
VEC_complex = ngs.VectorH1(mesh, complex=True)
VEC_real = ngs.VectorH1(mesh)

state, Kinv = solveStateComplex(fes)  # Is a GridFunction (field approximated on a fes)
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)

dJOmega = computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, 1, 1)

# Draw(state, radius=0.009)
# Draw(ngs.Norm(rot(state)), mesh, radius=0.009)
Draw(adjoint_inductance, radius=0.009)
Draw(adjoint_losses.imag, mesh, radius=0.009)  # Note that the imag part should be 0

In [ ]:
def SolveDeformationEquation(mesh, fX):
    VEC = ngs.VectorH1(
        mesh,
        order=1,
        dirichlety="coilHor|segment1",
        dirichletx="coilVert|segment2",
        dirichlet="arc",
        # definedon=mesh.Materials("core|air"),
    )
    PHI, X = VEC.TnT()

    # H1 dot product
    B = ngs.BilinearForm(VEC)
    B += ngs.InnerProduct(ngs.grad(X), ngs.grad(PHI)) * ngs.dx + ngs.InnerProduct(X, PHI) * ngs.dx
    B.Assemble()

    # Linear elasticity-type dot product (maybe try with lame parameter ?)
    # B = ngs.BilinearForm(VEC)
    # B += (
    #     ngs.InnerProduct(0.5 * (ngs.grad(X) + ngs.grad(X).trans), ngs.grad(PHI)) * ngs.dx
    #     + ngs.InnerProduct(X, PHI) * ngs.dx
    # )
    # B.Assemble()

    gfX = ngs.GridFunction(VEC)
    gfX.vec.data = -B.mat.Inverse(VEC.FreeDofs()) * fX.vec
    return gfX


# Without dirichlet conditions
dJ = ngs.GridFunction(VEC_real)
dJ.vec.FV().NumPy()[:] = dJOmega.vec.FV().NumPy()[:]
Draw(dJ, mesh, vectors={"grid_size": 40}, radius=0.009)

# With dirichlet conditions
descent_direction = SolveDeformationEquation(mesh, dJ)
# descent_direction.vec.FV().NumPy()[: (VEC_real.ndof // 2)] = np.zeros(VEC_real.ndof // 2)  # Cancel x coordinate
Draw(descent_direction.real, mesh, vectors={"grid_size": 20}, radius=0.009)

In [ ]:
def TaylorTest(cost, compute_adjoint, compute_shape_derivative):
    mesh = gen_mesh9(air_gap=4e-3, maxh=1e-3)
    fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
    VEC_complex = ngs.VectorH1(mesh, complex=True)
    VEC_real = ngs.VectorH1(mesh)
    a0, Kinv = solveStateComplex(fes)
    p0 = compute_adjoint(a0, Kinv)
    dJOmega = compute_shape_derivative(VEC_complex, VEC_real, a0, p0)

    direction = ngs.GridFunction(VEC_real)
    direction.Set(ngs.CF((0, 1)), definedon=mesh.Boundaries("front"))

    nb_sample = 7
    exponent_min = 3
    exponent_max = 10

    reminders = []

    X = []

    from mmglib import copy_ngmesh

    meshT = ngs.Mesh(copy_ngmesh(mesh.ngmesh))  # For transported mesh
    fesT = ngs.H1(meshT, order=1, dirichlet="arc|segment2|domainVert", complex=True)
    # VEC_complexT = ngs.VectorH1(meshT, complex=True)
    VEC_realT = ngs.VectorH1(meshT)

    for i in range(nb_sample):
        # Abscissa axis in log scale
        t = 10 ** (-(i / nb_sample * (exponent_max - exponent_min) + exponent_min))
        X.append(t)

        # Finite difference
        displacement = ngs.GridFunction(VEC_realT)
        displacement.Set(t * ngs.CF((0, 1)), definedon=mesh.Boundaries("front"))
        meshT.SetDeformation(displacement)
        aT, _ = solveStateComplex(fesT)
        difference = cost(aT, meshT) - cost(a0, mesh)

        # Reminder for analytic shape derivative
        reminder_value = np.abs(difference - ngs.InnerProduct(dJOmega.vec, t * direction.vec))
        reminders.append(reminder_value)

    Y = np.power(np.array(X), 2)
    fig, ax = plt.subplots(1, 1)

    ax.plot(X, reminders, label="reminder", marker="s")
    ax.plot(X, Y, label="y = x^2", linestyle="--", color="gray")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("t")
    ax.set_ylabel("Reminder")
    ax.set_title("Taylor test")
    ax.legend()
    ax.grid(True, which="both", ls="--")

    plt.tight_layout()
    plt.show()


TaylorTest(Inductance, solveAdjointInductance, computeInductanceShapeDerivative)
TaylorTest(Inductance, solveAdjointInductance, computeInductanceShapeDerivativeAuto)
TaylorTest(Losses, solveAdjointLosses, computeLossesShapeDerivative)
TaylorTest(Losses, solveAdjointLosses, computeLossesShapeDerivativeAuto)

In [ ]:
# Initializations
mesh = gen_mesh9(air_gap=4e-3, maxh=5e-3)
fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
VEC_complex = ngs.VectorH1(mesh, complex=True)
VEC_real = ngs.VectorH1(mesh)

l = 1e-2
b = 1e-1

# State and cost function
state, Kinv = solveStateComplex(fes)
constraint = Constraint(state, mesh)
Jold = CostFunction(state, l, b, constraint, mesh)

# Adjoints, shape derivate then descent direction
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)
dJOmega = computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, 1, 1)

descent_direction = SolveDeformationEquation(mesh, dJOmega)
currentNorm = ngs.Norm(descent_direction.vec)
derivative = ngs.InnerProduct(dJOmega.vec, descent_direction.vec)

if derivative > 0:
    raise ValueError("derivative positive :(")


def move_ngmesh_2D(displ, mesh):
    mat_displ = displ.vec.FV().NumPy()
    nb_points = len(mat_displ) // 2
    for i, point in enumerate(mesh.ngmesh.Points()):
        vx = mat_displ[i]
        vy = mat_displ[i + nb_points]
        point[0] += vx
        point[1] += vy
    mesh.ngmesh.Update()


from mmglib import run_adapt, copy_ngmesh

# print("Initial mesh")
# state, Kinv = solveStateComplex(fes)
# Draw(ngs.Norm(rot(state)), mesh)

# print("copy of Initial mesh")
# copied_ngmesh = copy_ngmesh(mesh.ngmesh)
# copied_mesh = ngs.Mesh(copied_ngmesh)
# Draw(ngs.Norm(rot(state)), copied_mesh)

print("Moved mesh")
descent_direction.vec.data = 2e-7 * descent_direction.vec
move_ngmesh_2D(descent_direction, mesh)
# mesh.ngmesh.OptimizeMesh2d() # Does not work

# Test the descent of the cost
fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
state, Kinv = solveStateComplex(fes)
constraint = Constraint(state, mesh)
Jnew = CostFunction(state, l, b, constraint, mesh)

if Jnew < Jold:
    print("J decreased")
else:
    print("J increased")
Draw(ngs.Norm(rot(state)), mesh)

# print("copy of Initial mesh")
# Draw(ngs.Norm(rot(state)), copied_mesh)

print("Optimized moved mesh")
new_ngmesh, return_code = run_adapt(mesh.ngmesh, hausd=3e-6, hmax=2e-3)
new_mesh = ngs.Mesh(new_ngmesh)
fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
state, Kinv = solveStateComplex(fes)
Draw(ngs.Norm(rot(state)), new_mesh)

# print("Recover initial mesh")
# descent_direction.vec.data = -descent_direction.vec
# move_ngmesh_2D(descent_direction, mesh)
# fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
# state, tangent_matrix = solveStateComplex(fes)
# Draw(ngs.Norm(rot(state)), mesh)

In [ ]:
# Initialize geometry
mesh = gen_mesh9(air_gap=1.78e-3, maxh=2e-3)
maxh = 2e-3
# mesh = ngs.Mesh("4_optimized_mesh.vol")

fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
VEC_complex = ngs.VectorH1(mesh, order=1, complex=True)
VEC_real = ngs.VectorH1(mesh, order=1)

# Initialize data arrays and plots
costs = []
losse_values = []
inductances = []
num_plots = 3
fig, axes, hdisplay = create_plots(num_plots)
curve_labels = [
    ["Cost Function (W)"],
    ["Losses Conductor (W)"],
    ["Inductance (mH)"],
]

l_values = []
b_values = []
num_plots = 2
figBis, axesBis, hdisplayBis = create_plots(num_plots)
curve_labels_bis = [
    ["l"],
    ["b"],
]

scene = Draw(ngs.Norm(rot(state)), mesh, radius=0.0001, center=(0, 0))

# Algorithmic parameters
iter_max = 2000
converged = False
iter = 0
minstep = 1e-10
maxstep = 0.5
step = maxstep
eps = 1e-10

# Augmented Lagragian parameters
l = 1e-2
b = 1e-1
btarget = 1e5

# Initializing state, adjoints, cost function
state, Kinv = solveStateComplex(fes)  # Is a GridFunction (field approximated on a fes)
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)
dJOmega = computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, l, b)
constraint = Constraint(state, mesh)
Jnew = CostFunction(state, l, b, constraint, mesh)

export_count = 0

# Optimization loop
while not converged and iter < iter_max:
    # Step 1: Update fespace
    fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
    VEC_complex = ngs.VectorH1(mesh, order=1, complex=True)
    VEC_real = ngs.VectorH1(mesh, order=1)

    # Step 2: Update the state and adjoint state
    state, Kinv = solveStateComplex(fes)
    adjoint_losses = solveAdjointLosses(state, Kinv)
    adjoint_inductance = solveAdjointInductance(state, Kinv)

    constraintold = Constraint(state, mesh)
    Jold = CostFunction(state, l, b, constraintold, mesh)

    # Step 2.5: Update the data arrays
    costs.append(Jold)
    losse_values.append(Losses(state, mesh))
    inductances.append(1e3 * Inductance(state, mesh))
    data = [
        [costs],
        [losse_values],
        [inductances],
    ]
    update_plots(fig, axes, hdisplay, data, curve_labels)

    l_values.append(l)
    b_values.append(b)
    data = [
        [l_values],
        [b_values],
    ]
    update_plots(figBis, axesBis, hdisplayBis, data, curve_labels_bis)

    # Step 2.5: Export if inductance cross 1mH
    if iter > 3 and np.sign(inductances[-1] - 1) != np.sign(inductances[-2] - 1):
        print(f"Sign change {iter=} {inductances[-1]=} {losse_values[-1]=}")
        mesh.ngmesh.Save(f"../outputs/5_optimized_mesh_{export_count}.vol")
        fig.savefig(f"../outputs/cv_{export_count}.png")
        figBis.savefig(f"../outputs/cv2_{export_count}.png")
        gf = ngs.GridFunction(ngs.L2(mesh))
        gf.Set(ngs.Norm(rot(state)))
        Draw(gf, mesh, radius=0.01, center=(0, 0), filename=f"../outputs/result_{export_count}.html")
        export_count += 1

    # Step 3: Compute shape derivative
    dJOmega = computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, l, b)

    # Step 4: Find a descent direction
    descent_direction = SolveDeformationEquation(mesh, dJOmega)
    # descent_direction.vec.FV().NumPy()[: (VEC_real.ndof // 2)] = np.zeros(VEC_real.ndof // 2) # Cancel x components
    currentNorm = ngs.Norm(descent_direction.vec)
    derivative = ngs.InnerProduct(dJOmega.vec, descent_direction.vec)

    if derivative > 1e-10:
        raise ValueError(f"derivative positive :( {derivative=}")

    # Step 4.5: Scene redraw
    # scene.Redraw(ngs.Norm(rot(state)), mesh, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))
    scene.Redraw(ngs.Norm(rot(state)), mesh, radius=0.001, center=(0, 0))
    # scene.Redraw(descent_direction, mesh, vectors={"grid_size": 40}, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

    # Step 5: Find suitable step with a line search
    Jold = Jnew
    copied_ngmesh = copy_ngmesh(mesh.ngmesh)
    copied_mesh = ngs.Mesh(copied_ngmesh)
    descent_direction_old = ngs.GridFunction(VEC_real)
    descent_direction_old.vec.data = descent_direction.vec
    i = 0
    imax = 7
    while i < imax:
        descent_direction.vec.data = step * maxh / (currentNorm + eps) * descent_direction_old.vec
        move_ngmesh_2D(descent_direction, mesh)

        return_code = 1

        # Mesh adaptation with mmg

        if iter % 10 == 0:
            # hausdist=(iter < 325) * 1e-5 + (415 > iter >= 325) * 1e-6 + (iter >= 415) * 1e-7
            # hausdist = 3e-7 + 1e-6 / (1 + np.exp(-4 / 30 * (iter - 150)))
            hausdist = 1e-6
            new_ngmesh, return_code = run_adapt(
                mesh.ngmesh,
                hausd=hausdist,
                hmax=2e-3,
            )
            mesh = ngs.Mesh(new_ngmesh)
            fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
            VEC_complex = ngs.VectorH1(mesh, order=1, complex=True)
            VEC_real = ngs.VectorH1(mesh, order=1)

        # Compute new cost
        state, Kinv = solveStateComplex(fes)
        constraint = Constraint(state, mesh)
        Jnew = CostFunction(state, l, b, constraint, mesh)

        if return_code == 1 and Jnew < Jold + (iter < 580) * np.abs(Jold):
            step = min(maxstep, 1.2 * step)
            break
        else:
            print(f"Line search fail")
            step = max(minstep, 0.5 * step)
            mesh = copied_mesh
            Jnew = Jold
        i += 1

    # Update augmented lagrangian parameter
    if i == imax:
        # print(f"Line search fail")
        pass
    else:
        # print(f"Line search success")
        l = l + b * Constraint(state, mesh)
        if b < btarget:
            b = min(1.1 * b, btarget)

    # Step 6: Stopping criteria
    if step <= minstep:
        print("Stoping criteria")
        converged = True
    # if np.abs(Jold - Jnew) <= 1e-4 * np.abs(Jold) and np.abs(constraint) <= 0.05 * L_target and b >= btarget:
    #     print("Stoping criteria")
    #     converged = True
    iter += 1

if iter == iter_max:
    print("Gradient descent max iteration reached")

plt.close(fig)

In [ ]:
M = np.array(inductances) - 1
sign_changes = np.where(np.sign(M[:-1]) != np.sign(M[1:]))[0]
print(sign_changes)

In [ ]:
print(f"{iter=}")

In [ ]:
mesh.ngmesh.Save("../outputs/5_optimized_mesh.vol")
fig.savefig("../outputs/cv.png")
figBis.savefig("../outputs/cv2.png")
gf = ngs.GridFunction(ngs.L2(mesh))
gf.Set(ngs.Norm(rot(state)))
Draw(gf, mesh, radius=0.01, center=(0, 0), filename="../outputs/result.html")